# 12 Special Methods for Sequences
Some notes, observations and questions along chapter 12.

### Vector Take #1: Vector2d Compatible
Best practice for a sequence constructor is to take the data as an iterable argument in the constructor, like all built-in sequence types do.

- could then be constructed as follows:

```
>>>Vector([3.1, 4.2])
Vector([3.1, 4.2])
>>> Vector((3, 4, 5))
Vector([3.0, 4.0, 5.0])
>>> Vector(range(10))
Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
```

In [19]:
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components) # array with the vector components

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components) # reprlib.repr() to get a limited-length representation of self._components
        components = components[components.find('['):-1] # array('d', prefix, and the trailing ) 
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

### Protocols and Duck Typing
- we don't need to inherit from a Sequence type in order to make a sequence
- we only need to implement the methods that fulfill the sequence protocol (duck typing)
- a protocol is an informal interface, defined only in documentation and not in code


- sequence types expect only the `__len__` and `__getitem__` methods
- for instance this is a sequence type, because it behaves like one:

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

### Vector Take #2: A Sliceable Sequence
- slicing in built-in sequence types: every one of them, when sliced, produces a new instance of its own type, and not of some other type

#### How Slicing Works

In [3]:
class MySeq:
    def __getitem__(self, index):
        # returns index for demonstration purpose
        return index

s = MySeq()
s[1] # returns a singe index

1

In [ ]:
s[1:4] # returns a slice object

slice(1, 4, None)

In [5]:
s[1:4:2] # start at 1, stop before 4, step by 2

slice(1, 4, 2)

In [6]:
s[1:4:2, 9] # now with the comma __getitem__ receives a tuple

(slice(1, 4, 2), 9)

In [7]:
s[1:4:2, 7:9] # we can return several slice objects as tuples

(slice(1, 4, 2), slice(7, 9, None))

In [8]:
dir(slice) # get all attributes of the build-in slice object

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

#### A Slice-Aware `__getitem__`
To make `__getitem__` more than functional than this:  
```
    def __getitem__(self, index):
        return self._components[index]
```
which would merely return an array of the chosen indices, we need to make it slice-aware.

This is what we need to add to our `Vector` class so its objects are sequences (and the `object[]` syntax handles slicing correctly be returning Vector-objects):

In [ ]:
import operator

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):              # If the key argument is a slice…
            cls = type(self)                    # …get the class of the instance (i.e., Vector) and…
            return cls(self._components[key])   # …invoke the class to build another Vector instance from a slice of the _components array.

        index = operator.index(key)             # utility in Python that converts its argument to an integer if possible without loss
        return self._components[index]

### Vector Take #3: Dynamic Attribute Access
- we want to build a way to access the first few components of a Vector object by getting its attributes using the `__getattr__` special method
- `__getattr__` method is invoked by the interpreter when attribute lookup fails
    - by default raises `AttributeError`, but we can modify it to do something else before:

In [ ]:
    __match_args__ = ('x', 'y', 'z', 't') #  allows positional pattern matching on the dynamic attributes supported in `__getattr__`

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name) # try to get the position of `name` in `__match_args__`.
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components): # pos is within range of the available components, return the component
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}' # otherwise raise AttributeError
        raise AttributeError(msg)

This is only invoked if the attribute is not found in the first place. Let's try it out:

In [ ]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components) # array with the vector components

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components) # reprlib.repr() to get a limited-length representation of self._components
        components = components[components.find('['):-1] # array('d', prefix, and the trailing ) 
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):              # If the key argument is a slice…
            cls = type(self)                    # …get the class of the instance (i.e., Vector) and…
            return cls(self._components[key])   # …invoke the class to build another Vector instance from a slice of the _components array.

        index = operator.index(key)             # utility in Python that converts its argument to an integer if possible without loss
        return self._components[index]
    
    """__match_args__ = ('x', 'y', 'z', 't') #  allows positional pattern matching on the dynamic attributes supported in `__getattr__`

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name) # try to get the position of `name` in `__match_args__`.
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components): # pos is within range of the available components, return the component
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}' # otherwise raise AttributeError
        raise AttributeError(msg)"""
    
        # --> this doesn't work; correction:
    
    def __getattr__(self, name):
        pos_names = ['x', 'y', 'z', 't']
        cls = type(self)
        if name in pos_names:
            pos = pos_names.index(name)
            if pos < len(self._components):
                return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

In [32]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [ ]:
v.x # attribute lookup, but x is not not an attribute explicitly set on v, so get into `__getattr__`
    # since x is in `pos_names`, it returns the value from the first position of the Vector object (0.0)

0.0

In [ ]:
v.x = 10 # here we assign the value 10 directly to the x-attribute of v (which is therefor newly created)

In [ ]:
v.x # it seems the new value is set, but ojo: `__getattr__` is no longer called to get it

10

In [35]:
v # but Vector object v still represents the values [0.0, 1.0, 2.0, 3.0, 4.0] because those values are in self._components

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

To fix this, we need to work on setting an attribute: very often when implementing `__getattr__`, we need to code `__setattr__` as well, to avoid inconsistent behavior in our objects.

In [36]:
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1: # special handling for single-character attribute names.
            if name in cls.__match_args__:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value) # default: __settattr__ of superclass (object)

- this forbids assigning values to single-letter attributes
- we are not disallowing setting all attributes, only single-letter, lowercase ones, to avoid confusion with the supported read-only attributes x, y, z, and t

### Vector Take #4: Hashing and a Faster ==
- we can use `functools.reduce()` to calculate a hash of a Vector object
    - the first argument to `functools.reduce()` is a two-argument function, and the second argument is an iterable
    - for instance `reduce(fn, lst)`, `fn` will be applied to the first pair of elements (`fn(lst[0], lst[1])`) producing a first result, `r1`; then `fn` is applied to `r1` and the next element `fn(r1, lst[2])` producing a second result, `r2`. Now `fn(r2, lst[3])` is called to produce `r3` … and so on until the last element, when a single result, `rN`, is returned

In [38]:
n = 0
for i in range(1, 6): 
    n ^= i # bitwise XOR operation: it compares two binary digits and returns 1 if they are different, 0 if they are the same

n

1

This is somehow used in:

In [39]:
import functools
import operator

functools.reduce(operator.xor, range(6))

1

Question: It is lacking here, why this would be allowed to calculate a unique hash. I can see that all the components of the vector are used to calculate the hash, but there would be so many other ways to do this. I suppose, since Fluent Python doesn't explain this, that most people learn this somewhere in the computer science curriculum ...

- Attempt of an answer: this method works well for sequences that need a composite hash, while balancing uniqueness, uniform distribution, and computational efficiency.

We can implement this to calculate the hash in `__hash__()`:

In [41]:
class Vector:
    typecode = 'd'

    # many lines omitted in book listing...

    def __eq__(self, other):  
        return tuple(self) == tuple(other)

    def __hash__(self):
        hashes = (hash(x) for x in self._components)  #  series of integer hashes for each component in the vector object
        return functools.reduce(operator.xor, hashes, 0)

A more efficient implementation of `__eq__()`:

In [42]:
    def __eq__(self, other):
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True

Or even more efficient, because the `all()` returns `False` right after the first mismatch; we don't need to go through all the elements:

In [43]:
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

### Vector Take #5: Formatting
- no notes here